# Execute the process and monitor the execution

To submit an execution request of a deployed process and monitor it, the OGC API Processes API uses the resource highlighted in the table below:

| **Resource**                   | **Path**                                  | **Purpose**                                                                     | **Part**   |
|--------------------------------|-------------------------------------------|---------------------------------------------------------------------------------|------------|
| Landing page                   | `/`                                       | Top-level resource serving as an entry point.                                   | Part 1     |
| Conformance declaration        | `/conformance`                            | Information about the functionality supported by the server.                    | Part 1     |
| API Definition                 | `/api`                                    | Metadata about the API itself.                                                  | Part 1     |
| Process list                   | `/processes`                              | Lists available processes with identifiers and links to descriptions.           | Part 1     |
| Process description            | `/processes/{processID}`                  | Retrieves detailed information about a specific process.                        | Part 1     |
| **Process execution**          | **`/processes/{processID}/execution`**    | **Executes a process, creating a job.**                                         | **Part 1** |
| Deploy Process                 | `/processes` (POST)                       | Deploys a new process on the server.                                            | Part 2     |
| Replace Process                | `/processes/{processID}` (PUT)            | Replaces an existing process with a new version.                                | Part 2     |
| Undeploy Process               | `/processes/{processID}` (DELETE)         | Removes an existing process from the server.                                    | Part 2     |
| EO Application Package         | `/processes/{processID}/package` (GET)    | Get the EOAP associated with a deployed process.                                | Part 2     |
| **Job status info**            | **`/jobs/{jobID}`**                       | **Retrieves the current status of a job.**                                      | **Part 1** |
| **Job results**                | **`/jobs/{jobID}/results`**               | **Retrieves the results of a job.**                                             | **Part 1** |
| Job list                       | `/jobs`                                   | Retrieves a list of submitted jobs.                                             | Part 1     |
| Job deletion                   | `/jobs/{jobID}`                           | Cancels and deletes a job.                                                      | Part 1     |

In [ ]:
import time
import json
from pystac.item_collection import ItemCollection
from ogc_api_client.api_client import Configuration
from ogc_api_client.api_client_wrapper import ApiClientWrapper
from typing import Dict, Optional
from ogc_api_client.models.status_info import StatusInfo, StatusCode

namespace = "acme"

# ogc_api_endpoint = f"http://zoo-project-dru-service/{namespace}/ogc-api"
ogc_api_endpoint = f"http://localhost:8080/{namespace}/ogc-api"

configuration = Configuration(
    host=ogc_api_endpoint,
)
client = ApiClientWrapper(configuration=configuration)

Define the input data for the execution:

In [ ]:
data = {
    "inputs": {
        "stac_items": [
            "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A",
            "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2A_10TFK_20220524_0_L2A"
        ],
        "aoi": "-121.399,39.834,-120.74,40.472",
        "epsg": "EPSG:4326",
        "bands": [
            "green",
            "nir"
        ]
    }
}


Set request headers

In [ ]:
headers = {
    "accept": "*/*",
    "Prefer": "respond-async;return=representation",
    "Content-Type": "application/json"
}

In [ ]:
process_id = "water-bodies"  # Replace with your process ID if different

# Submit the processing request
content = client.execute_simple(process_id=process_id, execute=data, _headers=headers)

if isinstance(content, StatusInfo):
    # Parse the response to get the job ID
    job_id = content.job_id
    print(f"Job submitted successfully. Job ID: {job_id}")
    status_location = next((link.href for link in content.links if link.rel == 'monitor'), None)
    if not status_location:
        status_location = f"{ogc_api_endpoint}/jobs/{job_id}"
                           
    print(f"Monitor job status at: {status_location}")
else:
    print(f"Failed to submit job. Status code: {response_data.status}")
    print("Response:", content.text)
    raise ValueError(f"Failed to submit job. Status code: {response_data.status}")


Monitor the Job Status

In [ ]:
print(f"\nMonitoring job status (job ID: {job_id})...")

while True:
    status = client.get_status(job_id=job_id)

    if status:
        print(f"Job status: {status.status}")
        
        # Check if the job is completed (either successful or failed)
        if status.status in [StatusCode.SUCCESSFUL, StatusCode.FAILED]:
            break
    else:
        print(f"Failed to get job status.")
        break
    
    # Wait for a few seconds before checking again
    time.sleep(10)

if status and status.status == StatusCode.SUCCESSFUL:
    # print(status)
    print("\nJob completed successfully. Retrieving results...")
    result = client.get_result(job_id=job_id)
    print(result)
    stac_feature_collection = result.get("stac_catalog").actual_instance.value.oneof_schema_2_validator
    print("STAC item collection:", stac_feature_collection)
else:
    print("\nJob did not complete successfully.")


In [ ]:
stac_items = ItemCollection.from_dict(stac_feature_collection).items

## Exploit the results

In [ ]:
for item in stac_items:

    print(item.get_assets())
    print(json.dumps(item.get_assets()["data"].to_dict(), sort_keys=True, indent=4))